In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import re
import os
from io import BytesIO
import docx
from PyPDF2 import PdfReader
from string import digits
from selenium import webdriver
from time import sleep
import keyboard
from bs4 import BeautifulSoup
import re

uni_names = list(set(pd.read_csv('uni_lst.csv')['University'].values.tolist()))

def read_word_file(file_path):
    doc = docx.Document(file_path)
    first_page_text = []
    for para in doc.paragraphs:
        first_page_text.append(para.text)
    return '\n'.join(first_page_text)
def download_wait(path_to_downloads=r'c:\Users\ethan\Downloads'):#mostly deprecated
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 60:
        sleep(1)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
        seconds += 1
    return seconds
def download_arxiv_old(arxiv_id):#mostly deprecated, just a backup method
    driver = webdriver.Chrome()
    driver.get(f'https://arxiv.org/pdf/{arxiv_id}.pdf')
    keyboard.press_and_release('ctrl + s')
    sleep(1)
    keyboard.write('temp.pdf')
    keyboard.press_and_release('enter')
    download_wait()
    driver.quit()
    return get_page(r"C:\Users\ethan\Downloads\temp.pdf")

def download_arxiv(arxiv_id):
    response = requests.get(f'https://arxiv.org/pdf/{arxiv_id}.pdf')

    if response.status_code == 200:
        pdf_content = BytesIO(response.content)
        #pdf_reader = PdfReader(pdf_content)
        return get_page(pdf_content,reader=True)
        
    else:
        return download_arxiv_old(arxiv_id)
def osf_download_old(publication_id):
    with open("temp.pdf", "wb") as pdf_file:
        pdf_file.write(requests.get(r'https://osf.io/download/'+publication_id).content)
    return get_page('temp.pdf')

def osf_download(publication_id):
    publication_link = r'https://osf.io/download/'+publication_id
    out=  get_page(BytesIO(requests.get(publication_link).content))
    if out ==[]:
        print('old')
        return osf_download_old(publication_link)
    return out

def get_page(file_name, reader=False):
    page = None
    try:
        reader  = PdfReader(file_name)#this should work either way as this function can take in either path or file
        page = reader.pages[0].extract_text()
    except:
        try:
            page = read_word_file(file_name)
        except:
            pass
    if not reader:
        os.remove(file_name)
    if page == None:
        return []
    return get_affiliations_and_emails(page)
def get_affiliations_and_emails(page):
    page = page.replace('\n', ' ').replace(',', ' ').replace('.com', '.com ')
    page = page.translate(str.maketrans('', '', digits))
    
    
    
    pattern =r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b'
    emails = re.findall(pattern, page)
    
    brute_force = []
    for name in uni_names:
        if name in page:
            brute_force.append(name)
    
    return list(set(emails) | set(brute_force))
    

    # page = page.replace('\n', ' ').replace(',', ' ').replace('.com', '.com ')
    # page = page.translate(str.maketrans('', '', digits))
    # pattern =r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b'
    # out = re.findall(pattern, page)

    # page_lst = page.split(' ')
    # for i in range(len(page_lst)):
    #     if page_lst[i].lower() in ['hospital','university','institute','school','academy', 'department', 'uc', 'tech', 'emory', ]:
    #         out.append(re.sub("[^a-zA-Z]+", "", ' '.join(page_lst[i-6:i+6]).lower()))
    # return list(set(out))
def get_authors(lst):
    out = []
    for auth in lst:
        out.append(auth['given']+ ' '+auth['family'])
    return out
def get_subject(lst):
    out = []
    for sub in lst:
        out.append(sub['text'])
    return out

In [4]:
pub = ""
def get_osf(num_days):
    OSF_token = 'XhAMbzE7SI0KcHYGLgn1oI5EeMqWbtiJuUOTjR1VdLDnF343kgT9OMhTwHiTY5lfd1ma1c'
    api_url = "https://api.osf.io/v2/preprints/"

    headers = {
        "Authorization": f"Bearer {OSF_token}"
    }

    params = {
            "page[size]": 100,# this is max ==100
            "filter[date_published][gte]":(datetime.now() - timedelta(days=num_days)).isoformat()
            
        }
    out = []
    i = 0
    while True:
        response = requests.get(api_url, headers=headers, params=params)
        data = response.json()
        print('here')
        for publication in data["data"]:
            paper_id = publication['id']
            # global pub
            # pub = publication
            #try:
            data_auths = requests.get(f'https://api.osf.io/v2/preprints/{paper_id}/citation/', headers=headers).json()['data']['attributes']['author']
            subjects = publication['attributes']['subjects'][0]
            title = publication["attributes"]["title"]
            abstract = publication['attributes']['description']
            url = publication['links']['html']
            date = publication['attributes']['date_published']
            emails = osf_download(publication['relationships']['primary_file']['links']['related']['href'][28:])
            #if i%10 ==0:
            print(i)
            i+=1
            out.append([paper_id, data_auths, subjects, title, abstract, url, date, emails])
            # except:
            #     pass
        try:
            api_url =data['links']['next']
            if api_url == None:
                break
        except:
            break
        #break#remove before full use, just for testing
    df = pd.DataFrame(out, columns=['paper_id', 'authors', 'subjects', 'title', 'abstract', 'url', 'date', 'affiliations'])
    df['authors'] = df['authors'].apply(get_authors)
    df['subjects'] = df['subjects'].apply(get_subject)
    df['source'] = 'OSF'
    return df
df = get_osf(2)


here
old
0
old
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
old
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
old
35
36
37
38
old
39
40
41
old
42
old
43
old
44
45
46
47
old
48
old
49
50
51
52
53
54
55
old
56
57
58
59
60
old
61
62
63
64
65
old
66
67
68
old
69
70
71
72
73
74
75
76
77
78
79
80
81
82
old
83
84
85
86
87
88
old
89
old
90
91
92
93
94
95
96
97
98
99
here
100
old
101
old
102
old
103
104
105
106
old
107
108
109
old
110
111
112
113
114
115
116
117
118
119
120
old
121
old
122
old
123
old
124
old
125
old
126
old
127
128
old
129
130
131
132
133
134
old
135
136
137
138
139
140
141
142
old
143
144
old
145
146
147
148
149
old
150
old
151
152
153
154
155
156
old
157
158
159
160
old
161
162
old
163
164
old
165
old
166
167
old
168
169
170
171
172
173
174
175
176
old
177
178
179
180
181
182
183
old
184
old
185
186
187
188
189
190
191
192
old
193
194
old
195
old
196
197
198
old
199
here
old
200
old
201
202
203
old
204
old
205
old
206
old
207
208
209
210
211
212
213
214
215
216
217
218
219

Multiple definitions in dictionary at byte 0xc6ff73 for key /Im187
Multiple definitions in dictionary at byte 0xc6ff82 for key /Im187
Multiple definitions in dictionary at byte 0xc6ff91 for key /Im187
Multiple definitions in dictionary at byte 0xd081fe for key /Im198
Multiple definitions in dictionary at byte 0xd0820d for key /Im198
Multiple definitions in dictionary at byte 0x13753cd for key /Im265
Multiple definitions in dictionary at byte 0x13753dc for key /Im265
Multiple definitions in dictionary at byte 0x13753eb for key /Im265
Multiple definitions in dictionary at byte 0x13e397a for key /Im292
Multiple definitions in dictionary at byte 0x13e3989 for key /Im292


old
419
420
old
421
old
422
old
423
424
425
old
426
old
427
428


XRef object at 1019668 can not be read, some object may be missing


old
429


...

XRef object at 1160514 can not be read, some object may be missing


old
431
432
433
434
435
436
437
438
439
440
old
441
442
old
443
444
445
446
447
448
449
450
451
452
old
453
454
455
old
456
457
458
459
460
old
461
462
463
old
464
465
466
467
468
old
469
470
471
472
473
old
474
old
475
476
old
477
478
479
480
481
old
482
483
old
484
485
old
486
487
old
488
old
489
490
491
492
493
494
495
496
497
498
499
here
500
501
502
503
504
505
506
507
508
509
510
511
512
old
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
old
529
530
old
531
532
533
534
old
535
536
old
537
538
539
540
old
541
542
543
old
544
545
546
old
547
old
548
549
550
551
552
553
554
555
556
557
558
old
559
560
old
561
old
562
old
563
old
564
old
565
566
567
568
569
570
571
572
573
574
575
576
577
old
578
579
580
581
582
583
old
584
585
old
586
old
587
588
589
590
591
592
593
594
595
596
597
598
599
here
old
600
old
601
old
602
old
603
old
604
old
605
606
607
old
608
609
610
611
old
612
old
613
old
614
old
615
old
616
617
old
618
old
619
old
620
621
old
622
old
623
624
625
62

Overwriting cache for 0 194


850
851
852
853
854
855
856
857
858
old
859
860
861
862
863
old
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
old
881
old
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
old
898
old
899
here
old
900
old
901
old
902
903
old
904
old
905
906
907
old
908
old
909
old
910
911
912
913
914
915
916
917
918
919
old
920
921
922
923
924
925
old
926
927
928
929
930
931
old
932
933
934
935
936


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [5]:
df.to_parquet('osf_data.parquet', index=False)

In [6]:
df['affiliations'].astype(str).nunique()

548

In [7]:
#2:52 on old system
# then 3:21
#brute force old 2:21
# brute force new 2:19

In [8]:
#3:09 old only
#new 4:02 try 1

In [9]:
# to do
# llama for abstract probably just for top 20 per user
# fuzzy search between stated wants and subjects
# then just need to make Plotly dash site
# also need to get Patent, trademarks, grants data as cake
# primary focus on grant db

# Now w/ ARXIV since OSF seems to miss it

In [2]:

def get_arxiv_catchup_per_subject(soup):
    print('here')
    out = []
    data = soup.text.split('\n arXiv:')
    for entry in data[1:]:
        entry = entry.split('\n\n')
        try:
            entry.remove("")
        except:
            pass
        try:
            entry.remove(" ")
        except:
            pass
        entry = [x for x in entry if not 'Comments: ' in x]

        try:
            entry_data = {
                'paper_id':entry[0].split(" ")[0],
                'title':entry[1].replace("Title: ", '').replace("\n", ''),
                'authors':entry[2].replace("Authors:", '').replace("\n", '').split(', '),
                'subjects':entry[3].replace("Subjects: ", '').replace("\n", ''),
                'abstract':entry[4].replace("\n", ''),
                'affiliations': download_arxiv(entry[0].split(" ")[0])
            }
            out.append(entry_data)
            print('ex')
        except:
            pass
        
    return out
def get_arxiv_per_day(date):
    out = []
    for subject in ['q-fin', 'cs', 'math', 'q-bio', 'hep-th', 'stat', 'econ', 'eess']: 
        url = f"https://arxiv.org/catchup?smonth={str(date.month)}&group=grp_&sday={str(date.day)}&archive={subject}&method=with&syear={str(date.year)}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        out.extend(get_arxiv_catchup_per_subject(soup))
    df = pd.DataFrame.from_records(out)
    df['date']= date.strftime('%Y-%m-%d')
    df['source'] = 'arXiv'
    return df
date = datetime.utcnow()-timedelta(days=1)
df1 = get_arxiv_per_day(date)
#df1.to_parquet('arxiv_data.parquet', index=False)
df1


here
ex
ex
ex
ex
ex
ex
ex
ex
ex
ex
ex
here


In [12]:
def get_arxiv_days_back(num_back):
    df = pd.DataFrame()
    date = datetime.utcnow()
    for i in range(num_back):
        df = pd.concat([df, get_arxiv_per_day(date-timedelta(days=i))])
        df.to_parquet('arxiv_data_long_temp.parquet', index=False)
    return df
df3 =get_arxiv_days_back(7)
df3.to_parquet('arxiv_data_long.parquet', index=False)

here
1115
1120
here
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
1220
1225
1230
1235
1240
1245
1250
1255
1260
1265
1270


Multiple definitions in dictionary at byte 0x2cdef for key /Rotate
Multiple definitions in dictionary at byte 0x2d077 for key /Rotate
Multiple definitions in dictionary at byte 0x2d141 for key /Rotate
Multiple definitions in dictionary at byte 0x2d20b for key /Rotate


1275
1280
1285
1290
1295
1300
1305
1310
1315
1320
1325
1330
1335
1340
1345
1350
1355
1360
1365
1370
1375
1380
1385
1390
1395
1400
1405
1410
1415
1420
1425
1430
1435
1440
1445
1450
1455
1460
1465
1470
1475
1480
1485
1490
1495
1500
1505
1510
1515
1520
1525
1530
1535
1540
1545
1550
1555
1560
1565
1570
1575
1580


incorrect startxref pointer(3)


1585
1590
1595
1600
1605
1610
1615
1620
1625
1630
1635
1640
1645
1650
1655
1660
1665
1670
1675
1680
1685
1690
1695
1700
1705
1710
1715
1720
1725
1730
1735
1740
1745
here
1750
1755
1760
1765
1770
1775
1780
1785
1790
1795
1800
1805
1810
1815
1820
1825
1830
1835
1840
1845
1850
1855
1860
1865
1870
1875
1880
1885
1890
1895
1900
1905
1910
1915
1920
1925
1930
1935
1940
1945
1950
1955
1960
1965
1970
1975
1980
1985
1990
1995
2000
2005
2010
2015
2020
2025
2030
2035
2040
2045
2050
2055
2060
2065
here
2070
2075
2080
2085
2090
here
2095
2100
2105
2110
2115
2120
2125
2130
2135
2140
2145
2150
2155
2160
2165
2170
here
2175
2180
2185
2190
2195
2200


...

4590
4595
4600
4605
4610
4615
here
4620
4625
4630
4635
4640
4645
4650
4655
4660
4665
4670
4675
4680
4685
here
4690
4695
4700
4705
4710
4715
4720


Multiple definitions in dictionary at byte 0x53fc1 for key /Rotate
Multiple definitions in dictionary at byte 0x54073 for key /Rotate


4725
here
4730
4735
4740
4745
4750
4755
4760
4765
4770
4775
4780
4785
4790
4795
4800
4805
4810
4815
4820
4825
4830
4835
4840
4845
4850
here
4855
4860
4865
4870
4875
4880
4885
4890
4895
4900
4905
4910
here
4915
4920
4925
4930
4935
4940
4945
4950
4955
4960
4965
4970
4975
4980
4985
4990
4995
5000
5005
5010
5015
5020
5025
5030
5035
5040
5045
5050
5055
5060
5065
5070
5075
5080
5085
5090
5095
5100
5105
5110
5115
5120
5125
5130
5135
5140
5145
5150
5155
5160
5165
5170
5175
5180
5185
5190
5195
5200
5205
5210
5215
5220
5225
5230
5235
5240
5245
5250
5255
5260
5265
5270
5275
5280
5285
5290
5295
5300
5305
5310
5315
5320
5325
5330
5335
5340
5345
5350
5355
5360
5365
5370
5375
5380
5385
5390
5395
5400
5405
5410
5415
5420
5425
5430
5435
5440
5445
5450
5455
5460
5465
5470
5475
5480
5485
5490
5495
5500
5505
5510
5515
5520
5525
5530
5535
5540
5545
here
5550
5555
5560
5565
5570
5575
5580
5585
5590
5595
5600
5605
5610
5615
5620
5625
5630
5635
5640
5645
5650
5655
5660
5665
5670
5675
5680
5685
5690
5695
5700


Multiple definitions in dictionary at byte 0xd22db for key /Rotate


5825
5830
5835
5840
5845
5850
5855
5860
5865
5870
5875
5880
5885
5890
5895
5900
5905
5910
5915
5920
5925
5930
5935
5940
5945
5950
5955
5960
5965
5970
5975
5980
5985
5990
5995
6000
6005
6010
6015
6020
6025
6030
6035
6040
6045
6050
6055
6060
6065
6070
6075
6080
6085
6090
6095
6100
6105
6110
6115
6120
6125
6130
6135
6140
6145
6150
6155
6160
6165
6170
6175
here
6180
6185
6190
6195
6200
6205
6210
6215
6220
6225
6230
6235
here
6240
6245
6250
6255
6260
6265
6270
6275
6280
6285
6290
6295
here
6300
6305
6310
6315
6320
6325
6330
6335
6340
6345
6350
6355
6360
6365
6370
6375
6380
6385
6390
6395
6400
6405
6410
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [16]:
df2 = pd.concat([df,df3])
df2.to_parquet('combined_data.parquet', index=False)
df2 = df2.drop_duplicates(subset='paper_id')
df2.to_parquet('de_duped.parquet', index=False)

In [17]:
df2

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source
0,k5ghb,[Setblon Tembang],"[Social and Behavioral Sciences, Psychology, O...",Pentingnya Konseling Eksistensial-Humanistik d...,Penelitian ini merupakan penelitian kulitatif ...,https://osf.io/k5ghb/,2023-09-22T15:11:44.880299,[],OSF
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF
...,...,...,...,...,...,...,...,...,...
1539,2308.06220,"[Noah D. Gade, Jordan Rodu]",Methodology (stat.ME); Machine Learning (stat.ML),Nonlinear Permuted Granger Causality,Granger causal inference is a contentious but ...,NaN,2023-09-19,[],arXiv
1545,2309.06305,"[Jacob Dorn, Luther Yap]",Econometrics (econ.EM); Methodology (stat.ME),Sensitivity Analysis for Linear Estimands,We propose a novel sensitivity analysis framew...,NaN,2023-09-19,[],arXiv
1546,2309.07180,"[Jennifer Park, Rochelle E. Tractenberg]",Other Statistics (stat.OT); Applications (stat...,How do ASA Ethical Guidelines Support U.S. Gui...,"In 2022, the American Statistical Association ...",NaN,2023-09-19,[],arXiv
1550,2203.01435,"[František Bartoš, Eric-Jan Wagenmakers]",Methodology (stat.ME),A general approximation to nested Bayes factor...,A staple of Bayesian model comparison and hypo...,NaN,2023-09-19,[],arXiv


In [3]:
download_arxiv_old('2308.06220')# this checks that selenium works on a given machine

['University of Virginia', 'ndge@virginia.edu', 'jsrq@virginia.edu']

In [18]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
nlp = spacy.load("en_core_web_sm")
path = r'C:\Users\ethan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.bin'
model = KeyedVectors.load_word2vec_format(path, binary=True)
def sentenceSimplifier(txt): 
    doc = nlp(txt)
    out = ""
    for token in doc:
        if token.pos_ in ['PROPN', 'VERB','NUM', 'NOUN']:
            out+=token.lemma_+" "
    return out[:-1]
def sentence2vec(txt):
    simplified_txt = sentenceSimplifier(txt)
    words = simplified_txt.split(" ")
    vecs_arr = np.array([model[word] for word in words if word in model]).sum(axis=0)/len(words)
    return  vecs_arr

2

In [21]:
df2['abstract_vec'] = df2['abstract'].apply(sentence2vec)
df2.to_pickle('de_duped_w_vec.pkl')
df2

c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/4948 [00:00<?, ?it/s]

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec
0,k5ghb,[Setblon Tembang],"[Social and Behavioral Sciences, Psychology, O...",Pentingnya Konseling Eksistensial-Humanistik d...,Penelitian ini merupakan penelitian kulitatif ...,https://osf.io/k5ghb/,2023-09-22T15:11:44.880299,[],OSF,"[0.054275375, -0.0053556077, 0.021748861, 0.02..."
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06..."
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990..."
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF,"[-0.0114671495, 0.09863281, -0.040132143, 0.10..."
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0..."
...,...,...,...,...,...,...,...,...,...,...
1539,2308.06220,"[Noah D. Gade, Jordan Rodu]",Methodology (stat.ME); Machine Learning (stat.ML),Nonlinear Permuted Granger Causality,Granger causal inference is a contentious but ...,NaN,2023-09-19,[],arXiv,"[0.017944157, -0.013425849, 0.02205308, 0.0719..."
1545,2309.06305,"[Jacob Dorn, Luther Yap]",Econometrics (econ.EM); Methodology (stat.ME),Sensitivity Analysis for Linear Estimands,We propose a novel sensitivity analysis framew...,NaN,2023-09-19,[],arXiv,"[-0.004429767, -0.0180908, 0.005612825, 0.0742..."
1546,2309.07180,"[Jennifer Park, Rochelle E. Tractenberg]",Other Statistics (stat.OT); Applications (stat...,How do ASA Ethical Guidelines Support U.S. Gui...,"In 2022, the American Statistical Association ...",NaN,2023-09-19,[],arXiv,"[-0.056426037, -0.045546602, 0.039102975, 0.08..."
1550,2203.01435,"[František Bartoš, Eric-Jan Wagenmakers]",Methodology (stat.ME),A general approximation to nested Bayes factor...,A staple of Bayesian model comparison and hypo...,NaN,2023-09-19,[],arXiv,"[0.028341988, -0.018754406, -0.007142254, 0.06..."


In [22]:
df5 = pd.read_pickle('de_duped_w_vec.pkl')
df5

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec
0,k5ghb,[Setblon Tembang],"[Social and Behavioral Sciences, Psychology, O...",Pentingnya Konseling Eksistensial-Humanistik d...,Penelitian ini merupakan penelitian kulitatif ...,https://osf.io/k5ghb/,2023-09-22T15:11:44.880299,[],OSF,"[0.054275375, -0.0053556077, 0.021748861, 0.02..."
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06..."
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990..."
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF,"[-0.0114671495, 0.09863281, -0.040132143, 0.10..."
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0..."
...,...,...,...,...,...,...,...,...,...,...
1539,2308.06220,"[Noah D. Gade, Jordan Rodu]",Methodology (stat.ME); Machine Learning (stat.ML),Nonlinear Permuted Granger Causality,Granger causal inference is a contentious but ...,NaN,2023-09-19,[],arXiv,"[0.017944157, -0.013425849, 0.02205308, 0.0719..."
1545,2309.06305,"[Jacob Dorn, Luther Yap]",Econometrics (econ.EM); Methodology (stat.ME),Sensitivity Analysis for Linear Estimands,We propose a novel sensitivity analysis framew...,NaN,2023-09-19,[],arXiv,"[-0.004429767, -0.0180908, 0.005612825, 0.0742..."
1546,2309.07180,"[Jennifer Park, Rochelle E. Tractenberg]",Other Statistics (stat.OT); Applications (stat...,How do ASA Ethical Guidelines Support U.S. Gui...,"In 2022, the American Statistical Association ...",NaN,2023-09-19,[],arXiv,"[-0.056426037, -0.045546602, 0.039102975, 0.08..."
1550,2203.01435,"[František Bartoš, Eric-Jan Wagenmakers]",Methodology (stat.ME),A general approximation to nested Bayes factor...,A staple of Bayesian model comparison and hypo...,NaN,2023-09-19,[],arXiv,"[0.028341988, -0.018754406, -0.007142254, 0.06..."


In [26]:
from langdetect import detect
def get_lang(x):
    try:
        return detect(x)
    except:
        return 'Fail'
df5['language'] = df5['abstract'].apply(get_lang)
df6 = df5[df5['language'] == 'en']
df6.to_pickle('de_duped_w_vec_en.pkl')
df6

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec,language
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06...",en
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990...",en
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF,"[-0.0114671495, 0.09863281, -0.040132143, 0.10...",en
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0...",en
5,mkszn,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",WDR45B is differentially expressed in central ...,Metastasis to the brain occurs in a significan...,https://osf.io/mkszn/,2023-09-22T14:20:59.006294,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.015696138, 0.116664164, 0.048419025, 0.048...",en
...,...,...,...,...,...,...,...,...,...,...,...
1539,2308.06220,"[Noah D. Gade, Jordan Rodu]",Methodology (stat.ME); Machine Learning (stat.ML),Nonlinear Permuted Granger Causality,Granger causal inference is a contentious but ...,NaN,2023-09-19,[],arXiv,"[0.017944157, -0.013425849, 0.02205308, 0.0719...",en
1545,2309.06305,"[Jacob Dorn, Luther Yap]",Econometrics (econ.EM); Methodology (stat.ME),Sensitivity Analysis for Linear Estimands,We propose a novel sensitivity analysis framew...,NaN,2023-09-19,[],arXiv,"[-0.004429767, -0.0180908, 0.005612825, 0.0742...",en
1546,2309.07180,"[Jennifer Park, Rochelle E. Tractenberg]",Other Statistics (stat.OT); Applications (stat...,How do ASA Ethical Guidelines Support U.S. Gui...,"In 2022, the American Statistical Association ...",NaN,2023-09-19,[],arXiv,"[-0.056426037, -0.045546602, 0.039102975, 0.08...",en
1550,2203.01435,"[František Bartoš, Eric-Jan Wagenmakers]",Methodology (stat.ME),A general approximation to nested Bayes factor...,A staple of Bayesian model comparison and hypo...,NaN,2023-09-19,[],arXiv,"[0.028341988, -0.018754406, -0.007142254, 0.06...",en


In [14]:
for subject in ['cs', 'math', 'q-bio', 'hep-th', 'stat', 'econ', 'eess', 'q-fin']:
    url = f"https://arxiv.org/catchup?smonth={str(date.month)}&group=grp_&sday={str(date.day)}&archive={subject}&method=with&syear={str(date.year)}"
    print(url)

https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=cs&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=math&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=q-bio&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=hep-th&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=stat&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=econ&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=eess&method=with&syear=2023
https://arxiv.org/catchup?smonth=9&group=grp_&sday=21&archive=q-fin&method=with&syear=2023


# Old does not work fully, misses articles

In [31]:
import sys
sys.exit('End of working code')

SystemExit: End of working code

C:\Users\ethan\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:


import requests
import datetime as dt
import xml.etree.ElementTree as ET

# Get the current date and time
now = dt.datetime.now()

week_ago = now - dt.timedelta(days=3)
articles = []
url = f'http://export.arxiv.org/api/query?search_query=all'

params = {
    "start": 0, 
    "max_results": 100,  
    "sortBy": "submittedDate",
    "sortOrder": "descending",
    'set':'cs'
}
i=0
run_again = True
while run_again:
    print(params['start'])
    resp = requests.get(url, params=params)
    # Parse the XML response
    ns = {'r': 'http://www.w3.org/2005/Atom'}
    root = ET.fromstring(resp.text)

    # Extract the article data

    for entry in root.findall('r:entry', namespaces=ns):
        i+=1
        authors = []
        for author in entry.findall('r:author', namespaces=ns):
            authors.append(author.find('r:name', namespaces=ns).text)
        paper_name = entry.find('r:title', namespaces=ns).text
        abstract = entry.find('r:summary', namespaces=ns).text
        date_published = entry.find('r:published', namespaces=ns).text
        
        if datetime.strptime(date_published[:10], "%Y-%m-%d") <= week_ago:
            print(date_published)
            run_again=False
            break
        affiliations = []
        link = entry.find('r:link', namespaces=ns).get('href')
        # if 'arxiv.org' in link:
        #     print('https://arxiv.org/pdf/'+link.split('/')[-1]+'.pdf')
        #     affiliations = download_arxiv('https://arxiv.org/pdf/'+link.split('/')[-1]+'.pdf')
        article = {
            'authors': authors,
            'title': paper_name,
            'abstract': abstract,
            'date_published': date_published,
            'link': link,
            'affiliations':affiliations
        }
        articles.append(article)
    params['start']+=params['max_results']
 
df = pd.DataFrame.from_records(articles)
df


In [ ]:
page = pdf_reader.pages[0].extract_text()
print(page)

In [ ]:
if 'All' in None:
    print('test')

In [ ]:
import re

def get_affiliations_and_emails(page):
    # Define the regex pattern for email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b'
    
    # Find all email addresses in the page using re.findall
    emails = re.findall(email_pattern, page)
    
    # Define a regex pattern to match affiliations
    affiliation_pattern = r'(?:\w+\s){0,6}\b(?:hospital|university|institute|school|academy|department|uc|tech)\b(?:\s\w+){0,6}'

    
    # Find all affiliation matches in the page using re.findall
    affiliations = re.findall(affiliation_pattern, page, re.IGNORECASE)
    
    # Remove duplicates by converting to a set and back to a list
    output_list = list(set(emails + affiliations))
    
    return output_list



page = """
Here are some emails: john.doe@example.com, alice.smith@example.org
This is a t website. UC berkley
Contact us: contact@university.edu
"""
emails = get_affiliations_and_emails(page)
print(emails)


In [ ]:
import re

page = """Here are some emails: john.doe@example.com, alice.smith@example.org This is a t website. UC berkley Contact us: contact@university.edu"""

pattern = r'((?:\S+\s*){0,4})(hospital|university|institute|school|academy|department|uc|tech)((?:\s*\S+){0,4})'
matches = re.findall(pattern, page.replace('\n', ' '), re.IGNORECASE)

print(matches)


In [ ]:
import re

page = """Here are some emails: john.doe@example.com, alice.smith@example.org This is a t website. UC berkley Contact us: contact@university.edu"""

pattern = r'((?:\S+\s*){0,4})(hospital|university|institute|school|academy|department|uc|tech)((?:\s*\S+){0,4})'
matches = re.findall(pattern, page, re.IGNORECASE)
out = []
for match in matches:
    before, keyword1, after = match
    full_match = f"{before.strip()} {keyword1} {after.strip()}".strip()
    out.append(full_match)
out

In [ ]:
import re
import spacy

# Sample text string
text_string = "I studied at the London School of Economics and later taught at Harvard University."

# Regex pattern matching
pattern = r'\b(at|from|in)?\s*([\w\s]*?(University|College|Institute|Academy|School)[\w\s]*?)\b'
regex_matches = re.findall(pattern, text_string, re.IGNORECASE)
regex_universities = [match[1] for match in regex_matches]

# NER using spaCy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text_string)
ner_universities = [ent.text for ent in doc.ents if ent.label_ == 'ORG' and any(keyword in ent.text for keyword in ['University', 'College', 'Institute', 'Academy', 'School'])]

# Combine results
final_universities = list(set(regex_universities) | set(ner_universities))
print(final_universities)


In [ ]:
import re
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Sample text string
text_string = "Students at UCLA, University of Kansas, and Ohio State have done excellent research Emory."

# Regex pattern matching
def get_affiliations_and_emails(page):
    page = page.replace('\n', ' ').replace(',', ' ').replace('.com', '.com ')
    page = page.translate(str.maketrans('', '', digits))
    
    pattern = r'\b(at|from|in)?\s*([\w\s]*?(University\s+of\s+\w+|U\.?[\s]?[A-Z][\.]?[\s]?[A-Z]?[\.]?[A-Z]?|U[A-Z]+|College|Institute|Academy|School|State|Tech|UC\s+\w+)[\w\s]*?)\b'
    regex_matches = re.findall(pattern, page, re.IGNORECASE)
    regex_universities = [match[1].strip() for match in regex_matches]
    doc = nlp(page)
    ner_universities = [ent.text.strip() for ent in doc.ents if ent.label_ == 'ORG' and any(keyword in ent.text for keyword in ['University', 'College', 'Institute', 'Academy', 'School', 'MIT', 'Tech', 'State'])]
    well_known_outliers = ['MIT', 'Caltech', 'Georgia Tech', 'UVA', "Emory"]  # Add more as needed
    present_outliers = [outlier for outlier in well_known_outliers if outlier.lower() in page.lower()]
    return list(set(regex_universities) | set(ner_universities) | set(present_outliers))
#%timeit 
get_affiliations_and_emails(text_string)

In [ ]:
def get_affiliations_and_emails(page):
    page = page.lower()  # Convert the page to lowercase for case-insensitive matching
    page = page.replace('\n', ' ').replace(',', ' ').replace('.com', '.com ')

    pattern = r'\b(at|from|in)?\s*([\w\s]*?(university\s+of\s+\w+|u\.?[a-z]?[\s]?[a-z]?[\s]?[a-z]?|u[a-z]+|college|institute|academy|school|state|tech|uc\s+\w+)[\w\s]*?)\b'
    regex_matches = re.findall(pattern, page)
    regex_universities = set(match[1].strip() for match in regex_matches)

    doc = nlp(page)
    ner_universities = set(ent.text.strip() for ent in doc.ents if ent.label_ == 'ORG' and any(keyword in ent.text.lower() for keyword in ['university', 'college', 'institute', 'academy', 'school', 'mit', 'tech', 'state']))

    well_known_outliers = {'mit', 'caltech', 'georgia tech', 'uva', 'emory'}  # Use a set for faster membership testing
    present_outliers = [outlier for outlier in well_known_outliers if outlier in page]

    return list(regex_universities | ner_universities | set(present_outliers))

# Example usage:
text_string = "Students at UCLA, University of Kansas, and Ohio State have done excellent research Emory."
%timeit get_affiliations_and_emails(page)

In [ ]:
def get_affiliations_and_emails(page):
    page = page.lower()  # Convert the page to lowercase for case-insensitive matching
    page = page.replace('\n', ' ').replace(',', ' ').replace('.com', '.com ')

    pattern = r'\b(at|from|in)?\s*([\w\s]*?(university\s+of\s+\w+|u\.?[a-z]?[\s]?[a-z]?[\s]?[a-z]?|u[a-z]+|college|institute|academy|school|state|tech|uc\s+\w+)[\w\s]*?)\b'
    regex_matches = re.findall(pattern, page)
    regex_universities = set(match[1].strip() for match in regex_matches)

    doc = nlp(page)
    ner_universities = set(ent.text.strip() for ent in doc.ents if ent.label_ == 'ORG' and any(keyword in ent.text.lower() for keyword in ['university', 'college', 'institute', 'academy', 'school', 'mit', 'tech', 'state']))

    well_known_outliers = {'mit', 'caltech', 'georgia tech', 'uva', 'emory'}  # Use a set for faster membership testing
    present_outliers = [outlier for outlier in well_known_outliers if outlier in page]

    return list(regex_universities | ner_universities | set(present_outliers))

# Example usage:
text_string = "Students at UCLA, University of Kansas, and Ohio State have done excellent research Emory."
#%timeit 
get_affiliations_and_emails(page)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [43]:
import requests

# Define the API endpoint URL
api_url = "https://api.grants.gov/v1/public/opportunities"

# Set up query parameters (you may need to adjust these)
params = {
    "page": 1,           # Page number
    "rows": 10,          # Number of rows per page
    "sortBy": "openDate" # Sorting criteria (you can change this)
}

# Make the API request
response = requests.get(api_url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    # Now you can work with the data as needed
    for opportunity in data["opportunityDetails"]:
        print(f"Title: {opportunity['title']}")
        print(f"Agency: {opportunity['agency']['name']}")
        print(f"Posted Date: {opportunity['postedDate']}")
        print(f"Application Due Date: {opportunity['closingDate']}")
        print("\n")
else:
    print(f"Request failed with status code {response.status_code}")


ConnectionError: HTTPSConnectionPool(host='api.grants.gov', port=443): Max retries exceeded with url: /v1/public/opportunities?page=1&rows=10&sortBy=openDate (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001BC5A69F3D0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))